<a href="https://colab.research.google.com/github/colinmcnamara/austin_langchain/blob/main/labs/LangChain_101/Misc/101-1-streamlit_ollama_llava.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install -q langchain streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.3 MB/s eta 0:00:00


In [8]:
%%writefile streaming_app.py
import streamlit as st
from langchain.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda
from operator import itemgetter
import base64
from langchain.schema import AIMessage, HumanMessage

st.title("Multimodal Chat")

llm = Ollama(model="bakllava")


def bind_and_run_llm(payload):
    image = payload["image"]
    prompt = payload["prompt"]
    bound = llm.bind(images=[image])
    return bound.invoke(prompt)


image_template = "{image}"
image_prompt = PromptTemplate.from_template(image_template)
prompt_template = "{question}"
prompt = PromptTemplate.from_template(prompt_template)

chain = (
    {"image": itemgetter("image"), "prompt": prompt} |
    RunnableLambda(bind_and_run_llm)
)


if "messages" not in st.session_state:
    st.session_state["messages"] = [
            (AIMessage(content="How can I help you?"), False)
            ]

if "uploaded_file" not in st.session_state:
    st.session_state["uploaded_file"] = None

for msg in st.session_state.messages:
    if not msg[1]:
        st.chat_message(msg[0].type).write(msg[0].content)
    else:
        st.chat_message(msg[0].type).image(msg[0].data, width=200)

if uploaded_file := st.sidebar.file_uploader("Upload an image file",
                                             type=["jpg", "png"]):
    if st.session_state.uploaded_file != uploaded_file:
        st.session_state.uploaded_file = uploaded_file
        st.session_state.messages.append(
                (HumanMessage(
                    content=uploaded_file.name,
                    data=uploaded_file
                    ), True))
        st.chat_message("user").image(uploaded_file, width=200)

if prompt := st.chat_input():
    st.session_state.messages.append((HumanMessage(content=prompt), False))
    st.chat_message("human").write(prompt)

    response = ""
    if uploaded_file is not None:
        data = uploaded_file.getvalue()
        b64 = base64.b64encode(data).decode()

        response = chain.invoke({"question": prompt, "image": b64})
    else:
        response = "Please upload an image first"

    st.session_state.messages.append((AIMessage(content=response), False))
    st.chat_message("assistant").write(response)

Overwriting streaming_app.py


### Download and run ollama

Below, we:
1. download the ollama binary
2. make it executable
3. start ollama in the background
4. download the hosted bakllava model

In [3]:
%%capture
!curl -L https://ollama.ai/download/ollama-linux-amd64 -o ollama
!chmod +x ollama
!./ollama serve &>/content/ollama_logs.txt &
!./ollama pull bakllava

### Start and background streamlit app

In [ ]:
!streamlit run streaming_app.py &>/content/logs.txt &

## Find the IP of your instance

In [5]:
!curl ipv4.icanhazip.com
!echo "Copy this IP into the webpage that opens below"

35.227.3.82
Copy this IP into the webpage that opens below


## Expose the Streamlit app on port 8501

In [ ]:
!npx localtunnel --port 8501

npx: installed 22 in 3.73s
your url is: https://common-mails-doubt.loca.lt
